In [1]:
import sys
import scipy.io
import numpy
import math
import csv
from operator import add

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

epsilon = 10e-6

import time
import copy

inf = 1e10




In [2]:
# The function returns a tuple: The first element is the set B; 
#                               The second is the number of each shape in B
def getB():
    n_frequency = [ num_fq_unit - num_shape_fq_unit[i] + 1 for i in range(num_of_shapes) ]
    n_time = [ num_tm_unit - num_shape_tm_unit[i] + 1 for i in range(num_of_shapes) ]
    n = [ int(a*b) for a,b in zip(n_frequency,n_time) ]
    
    B1 = range(n[0]) # indices of shape 1
    B2 = range(n[0],n[0]+n[1]) # indices of shape 2
    B3 = range(n[0]+n[1],n[0]+n[1]+n[2]) # indices of shape 3 
    B4 = range(n[0]+n[1]+n[2],n[0]+n[1]+n[2]+n[3]) # indices of shape 4
    B5 = range(n[0]+n[1]+n[2]+n[3], n[0]+n[1]+n[2]+n[3]+n[4]) # indices of shape 5
    
    B = B1 + B2 + B3 + B4 + B5
#     B = B1 + B2 + B3 + B4
    return [B, n]

In [3]:
time_length = 2*1e-3 # in s
frequency_length = 2000 # in kHz

subcarrier_bw = 15 # subcarrier bandwidth (kHz)
unit_tl = 0.125*1e-3 # in s
unit_fl = 15*12 # in kHz

num_fq_unit = int(frequency_length/unit_fl)
num_tm_unit = int(time_length/unit_tl)

num_latency = 5 # the number of services constrained by latency
num_capacity = 10-num_latency # the number of services to maximize capacity
num_service = num_latency + num_capacity # total number of services

tti = [ x * 1e-3 for x in [0.5, 0.25, 0.125, 0.125, 0.15] ]
num_of_shapes = len(tti)

subcarrier_num = [ 12, 24, 48, 48, 12 ] # one subcarrier is 15 kHz
shape_bw = [ n*subcarrier_bw for n in subcarrier_num ]
num_shape_fq_unit = [ int(bw/unit_fl) for bw in shape_bw ]
num_shape_tm_unit = [ int(tl/unit_tl) for tl in tti ] 

tau = [ t*time_length for t in [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] ]

tmp = getB()
B = tmp[0]
nB = tmp[1]

In [4]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

In [5]:
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

In [6]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, x) for x in conflict_PRB]

In [7]:
with open('DataVol_list.csv', 'rb') as f:
    datavol_csv = csv.reader(f)
    datavol_table = list(datavol_csv)
    datavol_table = [ map(float,x) for x in datavol_table] # convert to float
    datavol_table = map(list, zip(*datavol_table)) # transpose table

from scipy.interpolate import interp1d

# interplotation for datavol_table
x = range(1,31)
y1 = datavol_table[0] # for shape 1
y2 = datavol_table[1] # for shape 2
y3 = datavol_table[2] # for shape 3
y4 = datavol_table[3] # for shape 4
y5 = datavol_table[4] # for shape 5

datavol_interpld = [interp1d(x,y1), interp1d(x,y2), interp1d(x,y3), interp1d(x,y4), interp1d(x,y5)]

In [8]:
def shapeCapacity(PRB_shape, avg_snr): 
    if PRB_shape < 0 or PRB_shape > 4:
        print 'shape_index is out of range: valid index is between 0--4'
        return
    return spectral_efficiency[PRB_shape] * datavol_interpld[PRB_shape](avg_snr)  # capacity is in bit

In [9]:
def getPRBShape(PRB_index):
    if PRB_index < nB[0]:
        PRB_shape = 0
    elif PRB_index < nB[0]+nB[1]:
        PRB_shape = 1
    elif PRB_index < nB[0]+nB[1]+nB[2]:
        PRB_shape = 2
    elif PRB_index < nB[0]+nB[1]+nB[2]+nB[3]:
        PRB_shape = 3
    else:
        PRB_shape = 4
    return PRB_shape

In [10]:
def getr(snr_mat, tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in B:
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [11]:
def getrShape0(snr_mat, tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[0]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [12]:
def getrShape1(snr_mat, tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[1]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [13]:
def getrShape2(snr_mat, tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[2]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [14]:
def getrShape4(snr_mat, tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in range(nB[4]):
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [15]:
def getr(snr_mat,tau):

    r = [ [ 0 for k in K ] for b in B ]
    
    for b in B:
        PRB_shape = getPRBShape(b)
        
        shifted_PRB_index = b
        for i in range(PRB_shape):
            shifted_PRB_index -= nB[i]
        
        num_frequency = num_fq_unit - num_shape_fq_unit[PRB_shape] + 1
        num_time = num_tm_unit - num_shape_tm_unit[PRB_shape] + 1
    
        PRB_pos_column = shifted_PRB_index % num_time
        PRB_endtime = (PRB_pos_column+1)*tti[PRB_shape]
        
        PRB_row_start = shifted_PRB_index / num_time # starting from 0
        PRB_row_end = PRB_row_start + num_shape_fq_unit[PRB_shape] - 1
        
        for k in K:
            avg_snr = sum(snr_mat[i,k] for i in range(PRB_row_start,PRB_row_end+1) )/( PRB_row_end-PRB_row_start+1 )
            r[b][k] = shapeCapacity(PRB_shape, avg_snr)   
        for k in K:
            if tau[k] < PRB_endtime: # the latency of service k cannot be satisfied by PRB b
                r[b][k] = 0
        
    return r

In [16]:
def getAlpha(lam, r, q):
    alpha=[]
    for b in B:
        alpha_b = sum( lam[i]*a[b][i] for i in I ) 
        alpha.append(alpha_b)
    return alpha

In [17]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kc being computed.
# The columns in Kl of the returned matrix x are zero vectors.
def solveP2( lam, r, q):
    PRB_alloc = [ -1 for b in B ] # PRB_alloc[b] is the index of service that PRB b should be allocated to
                                  # PRB_alloc[b]=-1 means that PRB b is not allocated
        
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    alpha = getAlpha(lam, r, q)
    for b in B:
        tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
        if max(tmp_list) > 0:   # PRB is allocated only if r[b][k]-alpha[b] is positive
            PRB_alloc[b] = len(Kl) + numpy.argmax(tmp_list) # Kl is added such that the value of PRB_alloc[b] is 
                                                       # coherent with the corresponding indexed position in K
    
    # Convert PRB_alloc to matrix x
    for b in B:
        k = PRB_alloc[b] # indexed service
        if k >= 0: # indicating that PRB_alloc[b] != -1
            sol_x[b][k] = 1
    
    return sol_x

In [18]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kl being computed.
# The columns in Kc of the returned matrix x are zero vectors.
def solveP3( lam, r, q ):
    # create optimization model
    modelKl = Model('Integer Programming - Kl')
    modelKl.modelSense = GRB.MINIMIZE
    modelKl.setParam('OutputFlag', False) # slience output
    
    # create varialbes for modelKl:
    xKl = []
    for b in B:
        xKl_b = []
        for k in Kl:
            xKl_b.append(modelKl.addVar(vtype=GRB.BINARY))
        xKl.append(xKl_b)
    modelKl.update()
    
    # add constraints 
    for k in Kl:
        modelKl.addConstr( sum(r[b][k]*xKl[b][k] for b in B) >= q[k] )
    modelKl.update()

    # set objective function
    alpha = getAlpha(lam, r, q)
    modelKl.setObjective(
        sum( alpha[b]*xKl[b][k] for k in Kl for b in B )
    )
    
    # solve modelKl
    modelKl.optimize()
    
    # construct variables to be returned 
    sol_x = [ [ 0 for k in K ] for b in B ]
    for b in B:
        for k in Kl:
            sol_x[b][k] = int(xKl[b][k].x)
            
    return sol_x

In [19]:
def getUtility(r):
    utility = [ [ 0 for b in B ] for k in K ]
    block_size = [4,4,4,4]
    for k in K:
        for b in B:
            shape = getPRBShape(b)
            utility[k][b] = r[b][k]/block_size[shape]
    return utility

In [20]:
def assignBlock(utility, x, r, q):
    sol_x = copy.deepcopy(x)
    collision = [ False for b in B ]
    util_Kl = numpy.array(utility[:len(Kl) :])
    util_Kc = numpy.array(utility[len(Kl): :])
    
    for b in B:
        if sum(sol_x[b]) > 0:
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[b][p] == True:
                    collision[p] = True

    to_be_alloc = copy.deepcopy(Kl)
    while len(to_be_alloc)>0 and collision.count(False)>0:
        [service, block] = numpy.unravel_index(numpy.argmax(util_Kl),util_Kl.shape)
#         print (service,block), util_Kl[service][block]
        service_bit = sum(r[b][service]*sol_x[b][service] for b in B)
        if service_bit < q[service] and not collision[block]:
            sol_x[block][service] = 1
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[block][p] == True:
                    collision[p] = True
        if service_bit >= q[service] and service in to_be_alloc:
            to_be_alloc.remove(service)
            for b in B:
                util_Kl[service][b] = -inf
        if collision[block]==True:
            for k in Kl:
                util_Kl[k][block] = -inf

    while collision.count(False)>0:
        [service, block] = numpy.unravel_index(numpy.argmax(util_Kc),util_Kc.shape)
        service += len(Kl)
        if not collision[block]:
            sol_x[block][service] = 1
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[block][p] == True:
                    collision[p] = True
        if collision[block] == True:
            for k in Kc:
                util_Kc[k-len(Kl)][block] = -inf

    return sol_x

In [21]:
def LPBased(rho, r, q):
    model = Model('Linear Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output

    x = []
    for b in B:
        x_b = []
        for k in K:
            x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
        
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        x_count = [ [ x[b][k].x for b in B] for k in K ]
    else:
        x_count = [ [ 0 for b in B] for k in K ]
        return x_count
    for b in B:
        for k in K:
            if x[b][k].x < rho:
                x_count[k][b] = 0
    return x_count

In [22]:
def isFeasible(x, r, q):
    if (numpy.dot( numpy.dot(numpy.matrix(a).transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(numpy.matrix(r),numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<numpy.matrix(q)).tolist()[0].count(True)>0:
        return False
    return True

In [23]:
def optSolver(r, q):
    model = Model('Interger Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output
    model.setParam('TimeLimit', 3*60) # time_limit for gurobi
    
    sol_x = [ [ 0 for k in K ] for b in B ]

    x = []
    for b in B:
        x_b = []
        for k in K:
    #         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
            x_b.append(model.addVar(vtype=GRB.BINARY))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        for b in B:
            for k in K:
                sol_x[b][k] = x[b][k].x
            
    return sum( r[b][k]*sol_x[b][k] for k in Kc for b in B ), isFeasible(sol_x,r,q)

In [24]:
def fbSolver(r, q):
    model = Model('Interger Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output
    model.setParam('TimeLimit', 3*60) # time_limit for gurobi
    
    sol_x = [ [ 0 for k in K ] for b in B ]

    x = []
    for b in B:
        x_b = []
        for k in K:
    #         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
            x_b.append(model.addVar(vtype=GRB.BINARY))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        1
    )

    model.optimize()
    
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        return True
            
    return False

In [25]:
def filterX(x, r, q):
    filtered_x = copy.deepcopy(x)
    for k in Kl:
        service_bit = sum(r[b][k]*filtered_x[b][k] for b in B)
        rate_k = [ r[b][k]*filtered_x[b][k] for b in B ]
        while service_bit > q[k]:
#             print (k,b), rate_k[b]
            b = numpy.argmax(rate_k)
            if filtered_x[b][k] == 0 or rate_k[b] < 0: break
            if service_bit - rate_k[b]*filtered_x[b][k] >= q[k]:
                service_bit -= rate_k[b]*filtered_x[b][k]
                filtered_x[b][k] = 0
            rate_k[b] = -1
    return filtered_x

In [26]:
def LPLD(M, varphi, beta, gamma, mute, r, q):
    start_time = time.time()
    x_count_LP = LPBased(0,r,q)
    obj_LP =  sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0

    x_fix_LP = []
    for rho in numpy.arange(0,0.95,0.025):
        xx_LP = [ [ 1 if x_count_LP[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
        xx_LP = filterX(xx_LP,r,q)
        x_fix_LP.append(xx_LP)
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]
    for xx_LP in x_fix_LP:
        sol_x = assignBlock(x_count_LP, xx_LP, r, q)
        obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        fixed_pairs = sum(xx_LP[b][k] for k in K for b in B)
        allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
        if obj>best_obj_sofar and isFeasible(sol_x,r,q):
            best_obj_sofar = obj
            best_x_sofar = copy.deepcopy(sol_x)
        if not mute:
            print 'new solution - LP: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x,r,q), 'obj = ', obj  
    if not mute:
        print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar,r,q), ' obj=', best_obj_sofar          
    
    check_interval = 20
    
    for j in range(1, M+1): # k belongs to [1,100]
        
        xKc = solveP2(lam,r,q)
        xKl = solveP3(lam,r,q)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])
        alpha = getAlpha(lam, r, q)

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()
        if not mute:
            print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
        if j>0 and j%check_interval == 0:
            x_count_LD = [ [ x_count[k][b]/(j*1.0) for b in B] for k in K ]
            
            x_fix_LD = copy.deepcopy(x_fix_LP)
            for rho in numpy.arange(0,0.95,0.025):
                xx = [ [ 1 if x_count_LD[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
                xx = filterX(xx,r,q)
                x_fix_LD.append(xx)

            for xx in x_fix_LD:
                sol_x = assignBlock(x_count_LD, xx, r, q)
                obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
                fixed_pairs = sum(xx[b][k] for k in K for b in B)
                allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
                if obj>best_obj_sofar and isFeasible(sol_x,r,q):
                    best_obj_sofar = obj
                    best_x_sofar = copy.deepcopy(sol_x)
                if not mute:
                    print 'new solution - LD: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x,r,q), 'obj = ', obj  
            if not mute:
                print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar,r,q), ' obj=', best_obj_sofar

    if not mute:       
        print 'LD upper bound = ', best_dual_sofar
        print 'Solving LD costs', time.time()-start_time, 'seconds'
    return best_obj_sofar, isFeasible(best_x_sofar,r,q)

In [27]:
def Greedy(r,q):
    x_fix = [ [ 0 for k in K ] for b in B ]
    sol_x = assignBlock(utility,x_fix,r,q)
    fb = isFeasible(sol_x,r,q)
    obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B) if fb else 0
    return obj, fb

In [28]:
def LPRounding(r,q):
    x_count_LP = LPBased(0,r,q)
    x_fix = [ [ 0 for k in K ] for b in B ]
    sol_x = assignBlock(x_count_LP, x_fix, r, q)
    fb = isFeasible(sol_x,r,q)
    obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B) if fb else 0
    return obj, fb

In [29]:
def LPOnly(r,q):
    x_count_LP = LPBased(0,r,q)
    x_fix_LP = []
    for rho in numpy.arange(0,0.95,0.025):
        xx_LP = [ [ 1 if x_count_LP[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
        xx_LP = filterX(xx_LP,r,q)
        x_fix_LP.append(xx_LP)
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]
    for xx_LP in x_fix_LP:
        sol_x = assignBlock(x_count_LP, xx_LP , r, q)
        obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        fixed_pairs = sum(xx_LP[b][k] for k in K for b in B)
        allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
        if obj>best_obj_sofar and isFeasible(sol_x,r,q):
            best_obj_sofar = obj
            best_x_sofar = copy.deepcopy(sol_x)
            
    return best_obj_sofar, isFeasible(best_x_sofar,r,q)

In [30]:
def LDOnly(M, varphi, beta, gamma, mute, r,q):
    start_time = time.time()
    x_count_LP = LPBased(0,r,q)
    obj_LP =  sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]       
    
    check_interval = 20
    
    for j in range(1, M+1): # k belongs to [1,100]
        
        xKc = solveP2(lam,r,q)
        xKl = solveP3(lam,r,q)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])
        alpha = getAlpha(lam, r, q)

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()
        if not mute:
            print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
        if j>0 and j%check_interval == 0:
            x_count_LD = [ [ x_count[k][b]/(j*1.0) for b in B] for k in K ]
            
            x_fix_LD = []
            for rho in numpy.arange(0,0.95,0.025):
                xx = [ [ 1 if x_count_LD[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
                xx = filterX(xx,r,q)
                x_fix_LD.append(xx)

            for xx in x_fix_LD:
                sol_x = assignBlock(x_count_LD, xx , r ,q)
                obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
                fixed_pairs = sum(xx[b][k] for k in K for b in B)
                allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
                if obj>best_obj_sofar and isFeasible(sol_x,r,q):
                    best_obj_sofar = obj
                    best_x_sofar = copy.deepcopy(sol_x)
                if not mute:
                    print 'new solution - LD: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x), 'obj = ', obj  
            if not mute:
                print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar,r,q), ' obj=', best_obj_sofar
    if not mute:      
        print 'LD upper bound = ', best_dual_sofar
        print 'Solving LD costs', time.time()-start_time, 'seconds'
    return best_obj_sofar, isFeasible(best_x_sofar,r,q)

In [31]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [32]:
# import random
# sample_num = 200

# snr_mat_list = [ numpy.matrix( [ [random.uniform(5,30) for i in K ] for u in range(11) ] ) for h in range(sample_num) ] 

# for normalized_rate in [0.032,0.064,0.128]:
    
#     greedy_gap_list = []
#     rd_gap_list = []
#     LP_gap_list = []
#     LD_gap_list = []
#     LPLD_gap_list = []
    
#     for h in range(sample_num):
        
#         start_time = time.time()
        
#         snr_mat = snr_mat_list[h]
#         curr_r = getr(snr_mat, tau)
#         curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbit
        
#         (obj_opt, fb_opt) = optSolver(r=curr_r, q=curr_q)
#         (obj_rd, fb_rd) = LPRounding(r=curr_r, q=curr_q)
#         (obj_LP, fb_LP) = LPOnly(r=curr_r, q=curr_q)
#         (obj_LD, fb_LD) = LDOnly(M=200, varphi=5, beta=0.9, gamma=0.5, mute=True, r=curr_r, q=curr_q)
#         (obj_LPLD, fb_LPLD) = LPLD(M=200, varphi=5, beta=0.9, gamma=0.5, mute=True, r=curr_r, q=curr_q)
#         utility = getUtility(r=curr_r)
#         (obj_greedy, fb_greedy) = Greedy(r=curr_r,q=curr_q)
    
#         rd_gap_list.append( abs(obj_opt - obj_rd)/obj_opt )
#         LP_gap_list.append( abs(obj_opt - obj_LP)/obj_opt )
#         LD_gap_list.append( abs(obj_opt - obj_LD)/obj_opt )
#         LPLD_gap_list.append( abs(obj_opt - obj_LPLD)/obj_opt )
#         greedy_gap_list.append( abs(obj_opt - obj_greedy)/obj_opt )

        
#         print 'runnring time of iter ', h, ': ', time.time()-start_time, 'seconds'
#         print 'rate = ', normalized_rate, 'greedy_gap = ', greedy_gap_list[-1]
#         print 'rate = ', normalized_rate, 'rd_gap = ', rd_gap_list[-1], 'LP_gap = ', LP_gap_list[-1], 'LD_gap = ', LD_gap_list[-1], 'LPLD_gap = ', LPLD_gap_list[-1]
        

#     rd_gap = mean(rd_gap_list)
#     LP_gap = mean(LP_gap_list)
#     LD_gap = mean(LD_gap_list)
#     LPLD_gap = mean(LPLD_gap_list)
#     greedy_gap = mean(greedy_gap_list)
    
#     print 'rate = ', normalized_rate, 'rd_gap = ', rd_gap, 'LP_gap = ', LP_gap, 'LD_gap = ', LD_gap, 'LPLD_gap = ', LPLD_gap      
#     print 'rate = ', normalized_rate, 'greedy_gap = ', greedy_gap

       

In [33]:
snr_mat = numpy.matrix([[ 24.42130695,  27.01265871,  29.30201528,   7.13861208,
          16.47898114,  22.69447115,  21.74435108,  15.42748504,
          23.56379969,  26.33527111],
        [ 22.33190727,  26.99235666,   7.7562922 ,  10.54346664,
          15.52839329,  26.74788932,  27.38548404,  14.48918902,
          24.63719871,  19.63629382],
        [ 17.51724531,  23.37919956,  26.98579328,   9.35025715,
          22.32692102,  11.93421716,  10.23993204,  15.55815635,
          12.89071123,   9.24137262],
        [ 29.54918125,  12.07632449,  24.01538095,  28.15479741,
          29.75195359,  10.190799  ,  14.2045514 ,  17.49368559,
           9.88447589,   9.73392575],
        [ 25.0223455 ,  23.90227576,  17.44670836,   6.18379495,
          22.84289628,  16.09700852,  16.30535449,  10.56226018,
           9.46042129,  13.6695326 ],
        [ 21.11313716,   8.86805933,  19.54751904,   9.56340909,
          16.1966827 ,  15.93872269,  10.0835697 ,  13.92085506,
          20.56351892,  22.99725646],
        [ 28.40718882,  29.98014357,  28.94975632,  21.87469362,
          29.07119697,   9.87290022,   5.62181866,  12.02532803,
          12.41564751,  24.0018508 ],
        [ 12.06698104,  14.97052524,  26.20469541,  27.99274488,
          12.46333221,  19.99327262,  14.13906995,   5.9604763 ,
          19.18106013,  19.13080749],
        [ 15.64031319,  10.2268432 ,  15.02498064,  22.47818857,
          18.82352462,  29.82238321,  24.00776091,  14.3763043 ,
          27.72770816,  17.20611143],
        [  5.22465262,  14.95041822,  27.85268977,  28.10794329,
           8.00642958,  23.75710242,  29.02785938,  13.28771548,
          21.3414978 ,  24.2081503 ],
        [ 18.74784454,  12.16083034,   7.18346512,  29.38097387,
          23.59674388,  23.51032958,  28.13159251,  26.51752969,
          28.08712507,   5.84124966]])


In [34]:
normalized_rate = 0.256
curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ]
# B_inuse = B_entire

for dl in [0.125, 0.25, 0.5, 0.75, 1]:
    
    start_time = time.time()

    tau = [ t*time_length for t in [dl, dl, dl, dl, dl, 1.0, 1.0, 1.0, 1.0, 1.0] ]
    
    scalable_r = getr(snr_mat, tau)
    nonscalable0_r = getrShape0(snr_mat, tau)
    nonscalable1_r = getrShape1(snr_mat, tau)
    nonscalable2_r = getrShape2(snr_mat, tau)
    nonscalable4_r = getrShape4(snr_mat, tau)
    
    (obj_scalable_opt, fb_scalable_opt) = optSolver(r=scalable_r, q=curr_q)
    (obj_scalable_proposed, fb_scalable_proposed) = LPLD(M=200, varphi=5, beta=0.9, gamma=0.5, mute=True, r=scalable_r, q=curr_q)
    
    (obj_nonscalable0_opt, fb_nonscalable0_opt) = optSolver(r=nonscalable0_r, q=curr_q)
    (obj_nonscalable1_opt, fb_nonscalable1_opt) = optSolver(r=nonscalable1_r, q=curr_q)
    (obj_nonscalable2_opt, fb_nonscalable2_opt) = optSolver(r=nonscalable2_r, q=curr_q)
    (obj_nonscalable4_opt, fb_nonscalable4_opt) = optSolver(r=nonscalable4_r, q=curr_q)
    
    print 'runnring time: ', time.time()-start_time, 'seconds'

    print 'dl = ', dl, 'scalable_opt = ', obj_scalable_opt, 'feasibility = ', fb_scalable_opt
    print 'dl = ', dl, 'scalable_proposed = ', obj_scalable_proposed, 'feasibility = ', fb_scalable_proposed
    print 'dl = ', dl, 'nonscalable0_opt = ', obj_nonscalable0_opt, 'feasibility = ', fb_nonscalable0_opt
    print 'dl = ', dl, 'nonscalable1_opt = ', obj_nonscalable1_opt, 'feasibility = ', fb_nonscalable1_opt
    print 'dl = ', dl, 'nonscalable2_opt = ', obj_nonscalable2_opt, 'feasibility = ', fb_nonscalable2_opt
    print 'dl = ', dl, 'nonscalable4_opt = ', obj_nonscalable4_opt, 'feasibility = ', fb_nonscalable4_opt

runnring time:  48.2186908722 seconds
dl =  0.125 scalable_opt =  65220.3298321 feasibility =  True
dl =  0.125 scalable_proposed =  16089.8985508 feasibility =  True
dl =  0.125 nonscalable0_opt =  0.0 feasibility =  False
dl =  0.125 nonscalable1_opt =  0.0 feasibility =  False
dl =  0.125 nonscalable2_opt =  0.0 feasibility =  False
dl =  0.125 nonscalable4_opt =  43504.8530994 feasibility =  True
runnring time:  43.2481918335 seconds
dl =  0.25 scalable_opt =  65220.3298321 feasibility =  True
dl =  0.25 scalable_proposed =  15039.4577589 feasibility =  True
dl =  0.25 nonscalable0_opt =  28047.4296735 feasibility =  True
dl =  0.25 nonscalable1_opt =  34068.0175827 feasibility =  True
dl =  0.25 nonscalable2_opt =  24130.9562476 feasibility =  True
dl =  0.25 nonscalable4_opt =  43504.8530994 feasibility =  True
runnring time:  47.6460149288 seconds
dl =  0.5 scalable_opt =  65220.3298321 feasibility =  True
dl =  0.5 scalable_proposed =  14781.6268356 feasibility =  True
dl =  0.

In [35]:
# curr_q = [ q_elem * 1024 * 1.024 for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbit
# tau = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
# scalable_r = getr(snr_mat, tau)
# fbSolver(r =scalable_r ,q=curr_q)

In [36]:
# feasibility check

# import random
# sample_num = 100
# normalized_rate = 1.024

# lalala = 0
# snr_mat_list = []
# while len(snr_mat_list) < sample_num:
#     lalala += 1
#     snr_mat = numpy.matrix( [ [random.uniform(5,30) for i in K ] for u in range(11) ] )
#     curr_r = getr(snr_mat, tau)
#     curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbit
#     if fbSolver(r=curr_r, q=curr_q):
#         snr_mat_list.append(snr_mat)

# for normalized_rate in [1.024]:
    
#     count_opt = 0
#     count_greedy = 0
#     count_LP = 0
#     count_LD = 0
    
#     for h in range(sample_num):
        
#         start_time = time.time()
        
#         snr_mat = snr_mat_list[h]
#         curr_r = getr(snr_mat, tau)
#         curr_q = [ q_elem * 1024 * normalized_rate for q_elem in [ 1, 1, 1, 1, 1 ] ] # Kbit
#         utility = getUtility(r=curr_r)
        
#         if 

        
#         print 'runnring time of iter ', h, ': ', time.time()-start_time, 'seconds'
#         print 'rate = ', normalized_rate, 'greedy_gap = ', greedy_gap_list[-1]
#         print 'rate = ', normalized_rate, 'rd_gap = ', rd_gap_list[-1], 'LP_gap = ', LP_gap_list[-1], 'LD_gap = ', LD_gap_list[-1], 'LPLD_gap = ', LPLD_gap_list[-1]
        

#     rd_gap = mean(rd_gap_list)
#     LP_gap = mean(LP_gap_list)
#     LD_gap = mean(LD_gap_list)
#     LPLD_gap = mean(LPLD_gap_list)
#     greedy_gap = mean(greedy_gap_list)
    
#     print 'rate = ', normalized_rate, 'rd_gap = ', rd_gap, 'LP_gap = ', LP_gap, 'LD_gap = ', LD_gap, 'LPLD_gap = ', LPLD_gap      
#     print 'rate = ', normalized_rate, 'greedy_gap = ', greedy_gap

       